# 特征编码和数据预处理

# 思路
数据增强，例如把一些矩阵的一部分提取出来作为数据


模型设计思路
1. 特征工程
基因表达量：将FPKM或TPM值作为特征。
位置信息：将基因的染色体位置（包括起始和终止位置）考虑进来。这可以通过直接使用这些数值或将它们转换为基因长度等派生特征来实现。
2. 序列模型或图模型
序列模型：如果你认为基因在染色体上的顺序是重要的，可以考虑使用序列模型（如LSTM或Transformer）。这要求将每个细胞的基因按照染色体位置排序，然后用序列模型来处理排序后的序列。

图模型：考虑到基因之间可能存在的相互作用，可以使用图神经网络（GNN）来建模基因之间的关系。在这种情况下，每个基因可以作为图中的一个节点，而节点之间的边可以基于基因间的物理距离或已知的功能关联。

3. 自定义模型结构
组合模型：可以设计一个模型，该模型结合了传统的全连接层、序列处理层（如LSTM或Transformer层）和/或图层（如果使用GNN）。例如，可以先用一个序列或图模型处理基因表达数据和位置信息，然后将这些模型的输出连接到一个或多个全连接层以进行最终的分类。
数据准备
考虑到每个样本是一个单独的表格，你需要将这些表格转换为统一格式的数据，这可能意味着：

标准化基因列表：确保每个样本表格包含相同的基因集，这可能需要填充缺失的基因表达量（例如，使用0或基因表达量的平均值）。
特征编码：将染色体位置编码为可用的数值特征，例如通过计算基因的长度或将染色体编号转换为独热编码。
实现建议
数据预处理：编写脚本来预处理每个样本的数据，确保它们具有统一的格式和特征集。
模型选择：基于你对数据的理解和任务的需求，选择适当的模型架构。PyTorch提供了灵活的API来构建自定义模型，而Fastai可以简化训练流程。
训练和验证：训练模型时，可能需要特别注意防止过拟合，特别是当样本数量相对于特征数量较少时。使用适当的正则化技术和验证策略来评估模型性能。

In [18]:
import os
import pandas as pd
from collections import defaultdict

# 假定的数据根目录
data_root = 'E:\RNA-data\PRJNA494560'

In [24]:
total_subdirs = 0
subdirs_per_dir = {}

for root, dirs, files in os.walk(data_root):
    if root == data_root:  # 只在顶层目录计算子目录数量
        for dir in dirs:
            subdir_count = len([name for name in os.listdir(os.path.join(root, dir)) if os.path.isdir(os.path.join(root, dir, name))])
            subdirs_per_dir[dir] = subdir_count
            total_subdirs += subdir_count

print(f"Total subdirectories: {total_subdirs}")
print("Number of subdirectories in each directory:")
for dir, count in subdirs_per_dir.items():
    print(f"{dir}: {count}")


Total subdirectories: 148
Number of subdirectories in each directory:
Adrenal gland: 5
Bladder: 5
Brain (mixed white and gray matter): 8
CD138+ enriched bone marrow cells: 9
Cervix: 2
Cervix, endometrium: 2
Colon: 3
Colon, epithelium: 5
Esophagus: 10
Kidney: 7
Liver: 10
Lung: 8
Mammary gland: 5
Ovary: 4
Pancreas: 7
Prostate: 6
Skeletal muscle: 6
Skin: 6
Small intestine: 6
Stomach: 13
Thyroid gland: 6
Tonsil: 7
Uterus (myometrium): 2
Whole blood nuclear cells: 6


In [25]:
# 保存每个文件路径和对应组织类型的信息
sample_files = []

# 遍历数据目录，收集每个样本文件的路径
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('_gene_abundances.tsv'):
            file_path = os.path.join(root, file)
            sample_files.append(file_path)

len(sample_files)

148

In [22]:
# 处理质量低的数据

# import os

# # 存储低于20%的log文件
# low_percentage_logs = []

# # 遍历dataroot路径下的所有文件
# for root, dirs, files in os.walk(data_root):
#     for file in files:
#         # 检查文件是否为log文件
#         if file.endswith('.log'):
#             file_path = os.path.join(root, file)
#             # 读取文件的最后一行
#             with open(file_path, 'r') as f:
#                 last_line = f.readlines()[-1]
#                 # 提取最后一行中的百分比值
#                 percentage = float(last_line.split('%')[0])  # 假设百分比值在最后一行的形式为 'xx%'
#                 # 检查百分比值是否低于20%
#                 if percentage < 20:
#                     low_percentage_logs.append(file_path)

# import shutil

# # 目标目录
# target_dir = 'E:\\RNA-data\\less_than20'

# # 遍历低于20%的log文件
# for log in low_percentage_logs:
#     # 获取log文件的目录
#     log_dir = os.path.dirname(log)
#     # 创建目标目录，保留原来的路径结构
#     target_log_dir = os.path.join(target_dir, os.path.relpath(log_dir, data_root))
#     os.makedirs(target_log_dir, exist_ok=True)
    
#     # 移动log文件所在目录的所有文件到目标目录
#     for file_name in os.listdir(log_dir):
#         shutil.move(os.path.join(log_dir, file_name), target_log_dir)


# # 遍历低于20%的log文件
# for log in low_percentage_logs:
#     # 获取log文件的目录
#     log_dir = os.path.dirname(log)
#     # 删除原来的log文件所在的文件夹
#     shutil.rmtree(log_dir)

In [ ]:
# 初始化一个集合，用于存储第一个文件的基因唯一标识符
first_file = sample_files[0]
df_first = pd.read_csv(first_file, sep='\t')
unique_genes = set(df_first[['Gene ID', 'Start', 'End']].apply(tuple, axis=1))

# 遍历其余的文件
for file in sample_files[1:]:
    df = pd.read_csv(file, sep='\t')
    # 对每个文件，将'Gene ID', 'Start', 'End'列组合作为唯一标识符
    current_genes = set(df[['Gene ID', 'Start', 'End']].apply(tuple, axis=1))
    # 找出所有文件中这些唯一标识符的交集
    unique_genes = unique_genes.intersection(current_genes)




In [43]:
len(unique_genes)

60662

In [46]:
list(unique_genes)[0]

('ENSG00000269845', 21750747, 21753438)

In [ ]:
import pandas as pd

# Assuming unique_genes is a set of tuples, where each tuple is (Gene ID, Start, End)

for file in sample_files:
    # Read the file into a pandas DataFrame
    df = pd.read_csv(file, sep='\t')
    
    # Convert the 'Gene ID', 'Start', 'End' columns to a set of tuples
    file_genes = set(df[['Gene ID', 'Start', 'End']].itertuples(index=False, name=None))
    
    # Find the intersection of the file's genes and the unique genes
    common_genes = file_genes & unique_genes
    
    # Filter the DataFrame to only include the common genes
    df_common = df[df[['Gene ID', 'Start', 'End']].apply(tuple, axis=1).isin(common_genes)]
    
    # Sort the DataFrame
    df_sorted = df_common.sort_values(by=['Gene ID', 'Start', 'End'])
    
    # Write the sorted DataFrame back to the file
    df_sorted.to_csv(file.replace('_gene_abundances.tsv', '_gene_abundances_preprocessed.tsv'), sep='\t', index=False)


In [48]:
import os
import shutil

old_root = 'E:\\RNA-data\\PRJNA494560'
new_root = 'D:\\OneDrive - whu.edu.cn\\文档 - 人工智能与RNA剪接\\PRJNA494560'

# 遍历 old_root 下的所有文件
for dirpath, dirnames, filenames in os.walk(old_root):
    for filename in filenames:
        # 检查文件是否为预处理的 tsv 文件
        if filename.endswith('_abundances.tsv'):
            # 获取旧文件的完整路径
            old_file_path = os.path.join(dirpath, filename)
            # 创建新的文件路径
            new_file_path = old_file_path.replace(old_root, new_root)
            # 创建新文件路径中的所有目录
            os.makedirs(os.path.dirname(new_file_path), exist_ok=True)
            # 将文件复制到新的位置
            shutil.copy2(old_file_path, new_file_path)


In [53]:
import pandas as pd
import os

# 假设data_path是包含所有TSV文件的目录路径
data_path = 'E:\RNA-data\PRJNA494560'
# 保存每个文件路径和对应组织类型的信息
files = []

# 遍历数据目录，收集每个样本文件的路径
for root, dirs, filess in os.walk(data_path):
    for file in filess:
        if file.endswith('_preprocessed.tsv'):
            file_path = os.path.join(root, file)
            files.append(file_path)

# 初始化一个空的DataFrame来存储整合后的数据
all_data = pd.DataFrame()

# 初始化一个列表来存储每个样本的标签
labels = []

for file in files:
    # 根据你的文件结构调整路径和提取标签的方式
    file_path = os.path.join(data_path, file)
    label = file_path.split('\\')[-3]  # 假设标签存储在父目录名称中
    labels.append(label)
    
    # 读取TSV文件
    df = pd.read_csv(file_path, sep='\t', usecols=['Gene ID', 'TPM'])
    
    # 转换为每个基因一个列的形式
    df = df.pivot_table(index='Gene ID', values='TPM').T
    
    # 将当前样本的数据追加到总DataFrame中
    all_data = pd.concat([all_data, df], ignore_index=True, sort=False)

# # 用0填充缺失值
# all_data = all_data.fillna(0)

# 添加标签列
all_data['Label'] = labels


In [57]:
all_data.to_csv('all_data.tsv',sep='\t', index=False)